In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 14.9 MB/s 
     |████████████████████████████████| 596 kB 72.1 MB/s 
     |████████████████████████████████| 6.6 MB 68.7 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, ElectraForSequenceClassification, AdamW
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# GPU 사용
device = torch.device("cuda")

In [ ]:
# !nvidia-smi

Sun Jul  3 06:07:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
train_dataset = '/content/drive/MyDrive/CP1/CP1_koelectra_3label/train_data_list_3label.tsv'
test_dataset = '/content/drive/MyDrive/CP1/CP1_koelectra_3label/test_data_list_3label.tsv'

In [ ]:
train_dataset = pd.read_csv(train_dataset,delimiter='\t').reset_index()
test_dataset = pd.read_csv(test_dataset,delimiter='\t').reset_index()

In [ ]:
train_dataset.loc[(train_dataset['Emotion'] == -1), 'Emotion'] = 2
test_dataset.loc[(test_dataset['Emotion'] == -1), 'Emotion'] = 2

In [ ]:
train_dataset

,index,Sentence,Emotion
0,0,아 더빙.. 진짜 짜증나네요 목소리,2
1,1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,2,너무재밓었다그래서보는것을추천한다,2
3,3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,2
4,4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
201314,201314,스쿼트하는데 허벅지폭팔할거같아요,2
201315,201315,맨날 드시고 오시지.,2
201316,201316,야동 야구 동영상 을 많이 보지 않습니까?,0
201317,201317,아니야. 다 내 잘못같아. 상사가 하는 말중에 틀린 말이 하나도 없더라.,2


In [ ]:
train_dataset[train_dataset['Emotion']== 0]

,index,Sentence,Emotion
150012,150012,박병호가 에서 고전하는 이유도 설명되는 좋은 분석입니다,0
150013,150013,개니지하지마세요,0
150024,150024,인간인가 원숭이인가,0
150026,150026,130만원이면 아이패드 프로 128기가네,0
150028,150028,트랜스포머 되는 폰은 언제 만들거냐?,0
...,...,...,...
201264,201264,노조나 종북은 홍체인식으로 잡아내면 되겠네요.,0
201274,201274,우리가 기억해야 할 것은 박근혜가 투표에 의해 선출되었다는 겁니다.,0
201292,201292,이와중에 케이티 못찾은사람들 많어 ㅋㅋ,0
201301,201301,그거 괜찮겠는데?,0


In [ ]:
temp0 = train_dataset[train_dataset['Emotion']== 0]
temp1 = train_dataset[train_dataset['Emotion']== 1].sample(n=6500,random_state=7)
temp2 = train_dataset[train_dataset['Emotion']== 2].sample(n=6500,random_state=7)
train_dataset = pd.concat([temp0,temp1,temp2])

In [ ]:
train_dataset

,index,Sentence,Emotion
150012,150012,박병호가 에서 고전하는 이유도 설명되는 좋은 분석입니다,0
150013,150013,개니지하지마세요,0
150024,150024,인간인가 원숭이인가,0
150026,150026,130만원이면 아이패드 프로 128기가네,0
150028,150028,트랜스포머 되는 폰은 언제 만들거냐?,0
...,...,...,...
44513,44513,중학교 때 친구 집에서 비디오로 본 팝콘무비 그 이하의 영화,2
182661,182661,인사도 못하겠음.,2
119552,119552,아무리 결과가뻔해도출연자들의쓰레기짓거리들은 못보겠네요 이제부터라도 쌈빡하게 좀 써보세요,2
118257,118257,허접한 시나리오..영화를 망치다,2


In [ ]:
test_dataset[test_dataset['Emotion']== 0]

,index,Sentence,Emotion
50006,50006,공원가니 수두룩하던데 참 한심하다 ......,0
50027,50027,항상 힘들지 뭐.,0
50033,50033,알았어. 앞으론 짭새란 표현 안 쓸게.,0
50047,50047,이건 관심병을 넘어 거의 정신병 수준인데....,0
50061,50061,세계전쟁의 주역이 되겠네.,0
...,...,...,...
67074,67074,진정한 보수는 젊은이들이다.,0
67075,67075,에피타이저 푸짐하게 주는 집이면 정말 좋은 것 같은데?,0
67078,67078,그러게. 뉴스에 나올 법 하던데?,0
67079,67079,지금 밤인데??? 눈??? 안보임,0


In [ ]:
temp0 = test_dataset[test_dataset['Emotion']== 0]
temp1 = test_dataset[test_dataset['Emotion']== 1].sample(n=2000,random_state=7)
temp2 = test_dataset[test_dataset['Emotion']== 2].sample(n=2000,random_state=7)
test_dataset = pd.concat([temp0,temp1,temp2])

In [ ]:
test_dataset

In [ ]:
#tsv용
class NSMCDataset(Dataset):
  
  def __init__(self, csv_file):
    # 일부 값중에 NaN이 있음...
    self.dataset = csv_file.dropna(axis=0).copy()
    # 중복제거
    self.dataset.drop_duplicates(subset=['Sentence'], inplace=True)
    self.tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

    print(self.dataset.describe())
  
  def __len__(self):
    return len(self.dataset)
  
  def __getitem__(self, idx):
    row = self.dataset.iloc[idx, 1:3].values
    text = row[0]
    y = row[1]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=128,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = NSMCDataset(train_dataset)
test_dataset = NSMCDataset(test_dataset)

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/486 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

               index       Emotion
count   18607.000000  18607.000000
mean   120407.235127      1.039340
std     60380.644955      0.808001
min        59.000000      0.000000
25%     67379.000000      0.000000
50%    138082.000000      1.000000
75%    172805.000000      2.000000
max    201317.000000      2.000000
              index      Emotion
count   5886.000000  5886.000000
mean   40737.025144     1.016480
std    19998.368377     0.813167
min        9.000000     0.000000
25%    23218.250000     0.000000
50%    46855.500000     1.000000
75%    58005.000000     2.000000
max    67101.000000     2.000000


In [ ]:
test_dataset.__len__()

5886

In [ ]:
test_dataset.dataset

,index,Sentence,Emotion
50006,50006,공원가니 수두룩하던데 참 한심하다 ......,0
50027,50027,항상 힘들지 뭐.,0
50033,50033,알았어. 앞으론 짭새란 표현 안 쓸게.,0
50047,50047,이건 관심병을 넘어 거의 정신병 수준인데....,0
50061,50061,세계전쟁의 주역이 되겠네.,0
...,...,...,...
41197,41197,너무 지루 하고 주인공 너무 감성적이다 적들은 그냥죽이는데 지는 살려주고 다님 보호...,2
59718,59718,세상악행은모두하면서살아가는 놈이있다면그놈은벼락맞아죽겠지요 아마자식새끼들도모두벼락맞아...,2
49556,49556,그 대단했던 원작 말아먹기의 대표적인 작품.고인이 된 원작자를 욕되게 하지 마라.,2
6759,6759,신파극 그 자체 ㅇㅇㅇㅇ,2


# Create Model

In [ ]:
#https://huggingface.co/docs/transformers/model_doc/electra
#ElectraForSequenceClassification 예제 참조

In [ ]:
model = ElectraForSequenceClassification.from_pretrained("monologg/koelectra-small-v3-discriminator",num_labels=3).to(device) #클래스 개수 지정

# 한번 실행해보기
# text, attention_mask, y = train_dataset[0]
# model(text.unsqueeze(0).to(device), attention_mask=attention_mask.unsqueeze(0).to(device))

Downloading:   0%|          | 0.00/458 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.0M [00:00<?, ?B/s]

Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-small-v3-discriminator and are newly initialized

In [ ]:
model.num_labels

3

In [ ]:
# 모델 레이어 보기
model

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

# Learn

In [ ]:
epochs = 50
batch_size = 16

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-6)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True) #torch형식의 dataset으로 load
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=True)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
losses = []
accuracies = []

for i in range(epochs):
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_loader):
    optimizer.zero_grad() #https://algopoolja.tistory.com/55
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    loss = F.cross_entropy(y_pred, y_batch)
    loss.backward() #autograd를 사용하여 역전파 단계를 계산합니다. https://ropiens.tistory.com/29
    optimizer.step() 

    total_loss += loss.item() #손실이 갖고 있는 스칼라 값을 가져올 수 있습니다. https://tutorials.pytorch.kr/beginner/pytorch_with_examples.html

    _, predicted = torch.max(y_pred, 1) #텐서 배열(임베딩 벡터)에서 최대 값의 인덱스를 저장
    correct += (predicted == y_batch).sum() #해당 인덱스와 label이 같은 값인지 확인
    total += len(y_batch)

    batches += 1
    if batches % 100 == 0:
      print("Batch Loss:", total_loss, "Accuracy:", correct.float() / total)
  
  losses.append(total_loss)
  accuracies.append(correct.float() / total)
  print("Train Loss:", total_loss, "Accuracy:", correct.float() / total)

  0%|          | 0/1163 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Batch Loss: 109.82640409469604 Accuracy: tensor(0.3394, device='cuda:0')
Batch Loss: 219.4624457359314 Accuracy: tensor(0.3509, device='cuda:0')
Batch Loss: 329.03789615631104 Accuracy: tensor(0.3527, device='cuda:0')
Batch Loss: 438.28484213352203 Accuracy: tensor(0.3602, device='cuda:0')
Batch Loss: 547.1038627624512 Accuracy: tensor(0.3694, device='cuda:0')
Batch Loss: 655.3370527029037 Accuracy: tensor(0.3770, device='cuda:0')
Batch Loss: 762.9030578136444 Accuracy: tensor(0.3861, device='cuda:0')
Batch Loss: 869.1958837509155 Accuracy: tensor(0.3933, device='cuda:0')
Batch Loss: 973.234126329422 Accuracy: tensor(0.4055, device='cuda:0')
Batch Loss: 1076.3568017482758 Accuracy: tensor(0.4143, device='cuda:0')
Batch Loss: 1177.5864776968956 Accuracy: tensor(0.4226, device='cuda:0')
Train Loss: 1241.1999090909958 Accuracy: tensor(0.4266, device='cuda:0')


  0%|          | 0/1163 [00:00<?, ?it/s]

Batch Loss: 99.60914981365204 Accuracy: tensor(0.5019, device='cuda:0')
Batch Loss: 198.90740823745728 Accuracy: tensor(0.4966, device='cuda:0')
Batch Loss: 294.7458601593971 Accuracy: tensor(0.5106, device='cuda:0')
Batch Loss: 390.96370220184326 Accuracy: tensor(0.5164, device='cuda:0')
Batch Loss: 485.4989421367645 Accuracy: tensor(0.5191, device='cuda:0')
Batch Loss: 579.8166593313217 Accuracy: tensor(0.5221, device='cuda:0')
Batch Loss: 676.2479341030121 Accuracy: tensor(0.5203, device='cuda:0')
Batch Loss: 770.9392317533493 Accuracy: tensor(0.5221, device='cuda:0')
Batch Loss: 865.2035497426987 Accuracy: tensor(0.5248, device='cuda:0')
Batch Loss: 960.8271582126617 Accuracy: tensor(0.5244, device='cuda:0')
Batch Loss: 1055.0822983980179 Accuracy: tensor(0.5238, device='cuda:0')
Train Loss: 1113.2110831737518 Accuracy: tensor(0.5240, device='cuda:0')


...

Batch Loss: 28.862494215369225 Accuracy: tensor(0.9031, device='cuda:0')
Batch Loss: 59.316460747271776 Accuracy: tensor(0.8944, device='cuda:0')
Batch Loss: 85.71862665563822 Accuracy: tensor(0.8969, device='cuda:0')
Batch Loss: 113.70910300686955 Accuracy: tensor(0.8967, device='cuda:0')
Batch Loss: 140.3937510214746 Accuracy: tensor(0.8964, device='cuda:0')
Batch Loss: 168.92728375270963 Accuracy: tensor(0.8943, device='cuda:0')
Batch Loss: 196.45522213354707 Accuracy: tensor(0.8952, device='cuda:0')
Batch Loss: 227.1681566797197 Accuracy: tensor(0.8929, device='cuda:0')
Batch Loss: 257.128480900079 Accuracy: tensor(0.8920, device='cuda:0')
Batch Loss: 282.8360467888415 Accuracy: tensor(0.8916, device='cuda:0')
Batch Loss: 313.47715308889747 Accuracy: tensor(0.8902, device='cuda:0')
Train Loss: 331.2139228992164 Accuracy: tensor(0.8908, device='cuda:0')


  0%|          | 0/1163 [00:00<?, ?it/s]

Batch Loss: 25.384485626593232 Accuracy: tensor(0.9031, device='cuda:0')
Batch Loss: 53.74198083393276 Accuracy: tensor(0.8963, device='cuda:0')
Batch Loss: 80.27665354125202 Accuracy: tensor(0.8965, device='cuda:0')
Batch Loss: 107.501294599846 Accuracy: tensor(0.8942, device='cuda:0')
Batch Loss: 135.52812608145177 Accuracy: tensor(0.8938, device='cuda:0')
Batch Loss: 159.5306626353413 Accuracy: tensor(0.8957, device='cuda:0')
Batch Loss: 187.0425312910229 Accuracy: tensor(0.8971, device='cuda:0')
Batch Loss: 213.43348581902683 Accuracy: tensor(0.8984, device='cuda:0')
Batch Loss: 242.13460400141776 Accuracy: tensor(0.8972, device='cuda:0')
Batch Loss: 269.33426652662456 Accuracy: tensor(0.8974, device='cuda:0')
Batch Loss: 297.9318077005446 Accuracy: tensor(0.8965, device='cuda:0')
Train Loss: 314.99295933917165 Accuracy: tensor(0.8965, device='cuda:0')


  0%|          | 0/1163 [00:00<?, ?it/s]

Batch Loss: 27.736672734841704 Accuracy: tensor(0.8981, device='cuda:0')
Batch Loss: 55.89363076724112 Accuracy: tensor(0.8937, device='cuda:0')
Batch Loss: 83.95238274149597 Accuracy: tensor(0.8938, device='cuda:0')
Batch Loss: 109.18529218994081 Accuracy: tensor(0.8977, device='cuda:0')
Batch Loss: 138.26925901137292 Accuracy: tensor(0.8951, device='cuda:0')
Batch Loss: 161.28766361251473 Accuracy: tensor(0.8975, device='cuda:0')
Batch Loss: 187.95492754876614 Accuracy: tensor(0.8984, device='cuda:0')
Batch Loss: 215.08636742085218 Accuracy: tensor(0.8977, device='cuda:0')
Batch Loss: 243.72879412025213 Accuracy: tensor(0.8965, device='cuda:0')
Batch Loss: 272.32989874109626 Accuracy: tensor(0.8953, device='cuda:0')
Batch Loss: 300.37505496293306 Accuracy: tensor(0.8958, device='cuda:0')
Train Loss: 316.97858341783285 Accuracy: tensor(0.8956, device='cuda:0')


In [ ]:
losses, accuracies

([1241.1999090909958,
  1113.2110831737518,
  1060.286343216896,
  1029.4262273311615,
  996.8402494490147,
  970.4900725185871,
  939.3101328611374,
  908.9748420417309,
  886.50738042593,
  856.5750769078732,
  833.8239293396473,
  813.6924231946468,
  795.2000258266926,
  773.4663606286049,
  759.0476258397102,
  729.6225681751966,
  715.8969370573759,
  701.7009145021439,
  683.0975148230791,
  668.416686475277,
  648.1774101704359,
  638.5602422058582,
  623.2681143432856,
  610.9543468654156,
  594.4165359437466,
  579.5025128722191,
  571.6729992032051,
  551.611751370132,
  541.6973883807659,
  525.2940996512771,
  516.0970444530249,
  491.9460334032774,
  482.31985399127007,
  482.2601585574448,
  466.82886055111885,
  459.4353925436735,
  443.49379758536816,
  424.1038143336773,
  423.03669595345855,
  413.58731028065085,
  396.67363093420863,
  385.847956597805,
  374.4712448976934,
  363.1391706764698,
  357.5118390712887,
  350.9897578880191,
  334.0526709780097,
  331.213

테스트 데이터셋 정확도 확인하기

In [ ]:
model.eval()

test_correct = 0
test_total = 0

for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
  y_batch = y_batch.to(device)
  y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
  _, predicted = torch.max(y_pred, 1)
  test_correct += (predicted == y_batch).sum()
  test_total += len(y_batch)

print("Accuracy:", test_correct.float() / test_total)

  0%|          | 0/368 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Accuracy: tensor(0.7258, device='cuda:0')


In [ ]:
# 모델 저장하기
path = '/content/drive/MyDrive/CP1/'
torch.save(model.state_dict(), path + "model_koelectra_label3_4.pt")

# 테스팅

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

In [ ]:
# path = '/content/drive/MyDrive/CP1/'
model.load_state_dict(torch.load('/content/drive/MyDrive/CP1/model_koelectra_label3.pt'))

<All keys matched successfully>

In [ ]:
def predict(text):
    encoded_review = tokenizer.encode_plus(
        text,
        return_tensors='pt',
        truncation=True,
        max_length=128,
        pad_to_max_length=True,
        add_special_tokens=True
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    # print(output)
    _, prediction = torch.max(output[0], dim=1)
    # print(torch.max(output[0], dim=1))
    print('입력 텍스트: ' + text)
    if str(prediction) == "tensor([1], device='cuda:0')":
        print('감정: 긍정적')
    elif str(prediction) == "tensor([2], device='cuda:0')":
        print('감정: 부정적')
    else:
        print('감정: 중립')

In [ ]:
predict('버핏은 이렇게 답했다')

입력 텍스트: 버핏은 이렇게 답했다
감정: 중립


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('안녕하세요')

입력 텍스트: 안녕하세요
감정: 중립


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('하.....승민아  널 지지한걸 후회하게만드는 발언은하지마  \n그동안 가던데로  너만의길을  올바르게 가라')

입력 텍스트: 하.....승민아  널 지지한걸 후회하게만드는 발언은하지마  
그동안 가던데로  너만의길을  올바르게 가라
감정: 중립


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('지가 배신때려서 박근혜대통령 탄핵시작됐었으면서...  이박쥐같은새끼는 절대안됨')

입력 텍스트: 지가 배신때려서 박근혜대통령 탄핵시작됐었으면서...  이박쥐같은새끼는 절대안됨
감정: 부정적


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('행복하네요')

입력 텍스트: 행복하네요
감정: 중립


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('잘하고 있다. 어디에 내 놔도 손색없다. 차원이 다르다.')

입력 텍스트: 잘하고 있다. 어디에 내 놔도 손색없다. 차원이 다르다.
감정: 부정적


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('윤석열이 대통령실이든 구태들이든 컨트롤 못 한다는게 맞는거 같다')

입력 텍스트: 윤석열이 대통령실이든 구태들이든 컨트롤 못 한다는게 맞는거 같다
감정: 중립


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('아니 진짜 개재미없네')

입력 텍스트: 아니 진짜 개재미없네
감정: 부정적


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('와 진짜 재미있다')

입력 텍스트: 와 진짜 재미있다
감정: 긍정적


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
predict('기분 좋네요 ㅎㅎ')

입력 텍스트: 기분 좋네요 ㅎㅎ
감정: 긍정적


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
li = ['두사람 사면시키려면 우리나라 도둑놈들 다 사면시켜야됨',
 '유승민은 바보다.\n자신을픽밥한 박통을 사면하자고.\n이게 진심 큰정치다 .',
 '국민에게 고통을 주는 갈라치기정치에서 국민통합이라는 메세지. 오십보백보 같은 전.현정권의 행보에서의 형평성의 논리로 보여지네',
 '집권 여당의 꾀에 넘어가지마라. 어차피 고생한거 1년 더 고생하고 너희들이 정권잡고 사면시켜라. 그리고 현정부는 심판하고!!',
 '국민통합? 국민통합을 위해서라면 큰 죄를 지은 사람을 봐줘야 한다는 건가요? 동의할 수 없습니다.',
 '그래도 솔직하시네',
 '유승민이 항상 실패했던 이유',
 '하.....승민아  널 지지한걸 후회하게만드는 발언은하지마  \n그동안 가던데로  너만의길을  올바르게 가라',
 '절대 안된다.',
 '유승민 넌 정치 하지마라..국민들 마음도 못 읽으면서 무슨 정치를 한다고 나대는 거냐?..',
 '법지키며 사는 국민을 개 돼지 취급하는 사면! 법 안지키고 살면 편한거 모르는 사람없다! 국민통합? 웃기고 자빠졌네! 생계형 사면이라면 모를까 저런 사면은 개나 줘라',
 '승민이형도...대통보다는 시장도전이 현실적일듯...',
 '이새\n\n\n끼. 지가 배신때려서 박근혜대통령 탄핵시작됐었으면서...  이박쥐같은새\n\n\n\n끼는 절대안됨. 한번배반때린건 또 그짓함.',
 '유승민 요 OOO  내통했구나 서로 짜짝쿵이 잘 맞네... 한쪽에선 제안하고 한쪽에서 환영하고..',
 '유승민이랑 김무성이 탄핵시켰자나 저거도 완장놀이 재밌어해서 큰일임',
 '하여간 정치권이란',
 '질문도 좋고 먼저 말 거는 모습도 좋고..나토에서 좋은 모습 보여줘서 너무 좋습니다!',
 '잘하고 있네.',
 '정수기랑 혜경궁 보단 나음',
 '이왕 영부인 된 거 그냥 좋게 봐주자 방구석에 처박혀 아무것도 안하고 있는거 보다 영부인 위치에서 자기가 할수 있는 뭐라도 해보겠다고 하는게 좋아 보임',
 '세계에 내놓으니 우아하고 귀족적이다. 관상보는분이 귀한상이라고 하더니 바로 표나네요. ',
 '잘하고 있다. 어디에 내 놔도 손색없다. 차원이 다르다.']

In [ ]:
text = ['관종 김건희 방치하고 대통령실 꼴 보면 자업자득임',
'민좆에 병신 하나 더 들어가서 나쁠것 없지 ㅋㅋㅋㅋㅋ 그래봤자 이재명 꼬붕 하나 더 늘어날 뿐',
'이준석은 언론 어떻게 다루는지 잘 모르는거 같음.',
'윤석열이 대통령실이든 구태들이든 컨트롤 못 한다는게 맞는거 같다',
'이준석 아니었으면 국힘 지지도 나락갔을테고  이번 대선도 나락이었을텐데  꼭 이렇게 이준석을 힘들게 해야 함??   "당 대표 중심으로 협력해라!"  이 말 한마디 하기가 글케 어려움??   진짜 실망스럽다',
'실망스럽다',
'이준석 요즘 너무 잘하는거 같지 않음?',
'이준석 요즘 잘하는거 같음']

In [ ]:
%%time
for i in text:
    print(predict(i))

입력 텍스트: 관종 김건희 방치하고 대통령실 꼴 보면 자업자득임
감정: 부정적
None
입력 텍스트: 민좆에 병신 하나 더 들어가서 나쁠것 없지 ㅋㅋㅋㅋㅋ 그래봤자 이재명 꼬붕 하나 더 늘어날 뿐
감정: 부정적
None
입력 텍스트: 이준석은 언론 어떻게 다루는지 잘 모르는거 같음.
감정: 부정적
None
입력 텍스트: 윤석열이 대통령실이든 구태들이든 컨트롤 못 한다는게 맞는거 같다
감정: 부정적
None
입력 텍스트: 이준석 아니었으면 국힘 지지도 나락갔을테고  이번 대선도 나락이었을텐데  꼭 이렇게 이준석을 힘들게 해야 함??   "당 대표 중심으로 협력해라!"  이 말 한마디 하기가 글케 어려움??   진짜 실망스럽다
감정: 부정적
None
입력 텍스트: 실망스럽다
감정: 부정적
None
입력 텍스트: 이준석 요즘 너무 잘하는거 같지 않음?
감정: 부정적
None
입력 텍스트: 이준석 요즘 잘하는거 같음
감정: 긍정적
None
CPU times: user 123 ms, sys: 7.93 ms, total: 131 ms
Wall time: 127 ms


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
